In [1]:
import sys
sys.path.append('../build')   # assume an out-of-source build

import numpy as np
import exafmm as fmm
import exafmm.helmholtz as helmholtz

In [2]:
helmholtz.__doc__

"exafmm's submodule for Helmholtz kernel"

### Create sources and targets

In [3]:
nsrcs = 100000
ntrgs = 100000

# generate random positions for particles
src_coords = np.random.random((nsrcs, 3))
trg_coords = np.random.random((nsrcs, 3))

# generate random charges for sources
src_charges = np.zeros(nsrcs, dtype=np.complex_)
src_charges.real = np.random.random(nsrcs)
src_charges.imag = np.random.random(nsrcs)

In [4]:
# create a list of source instances
sources = helmholtz.init_sources(src_coords, src_charges)

# create a list of target instances
targets = helmholtz.init_targets(trg_coords)

### Create an LaplaceFMM instance

In [5]:
fmm = helmholtz.HelmholtzFMM()
fmm.p = 10
fmm.nsurf = 6*(fmm.p-1)*(fmm.p-1) + 2
fmm.ncrit = 400
fmm.depth = 4
fmm.wavek = 10

### Setup FMM

Given sources, targets and FMM configurations, `setup()` builds the tree and interaction list and pre-compute invariant matrices.

Set `skip_P2P` to `True` to skip near-field interactions for BEM applications.

In [6]:
skip_P2P = False
tree = helmholtz.setup(sources, targets, fmm, skip_P2P)

### Evaluate

`evaluate()` triggers the evaluation and returns a $n_{trg} \times 4$ `numpy.ndarray`.
The $i$-th row of `trg_values` starts with the potential value of the $i$-th target, followed by its three gradient values.

In [7]:
trg_values = helmholtz.evaluate(tree, fmm)

P2M                  : 7.1251900e-01
M2M                  : 3.7610100e-01
P2L                  : 1.0300000e-04
M2P                  : 2.4810000e-03
P2P                  : 4.6419700e-01
M2L                  : 1.8297730e+00
L2L                  : 3.3043100e-01
L2P                  : 7.7763700e-01


In [8]:
trg_values[6]

array([-1004.07354653  -60.25727442j,   751.05914129+1040.77856515j,
       -1764.52385118 +715.31193763j,  4312.75412871-3396.27169089j])

### Check Accuracy

`fmm.verify(tree.leafs)` returns L2-norm the relative error of potential and gradient in a list. It only works when `skip_P2P` is set to `False`.

In [9]:
fmm.verify(tree.leafs)

[6.846847632469516e-08, 1.1168061828349136e-06]

### Update charges of sources and run FMM iteratively

In [10]:
niters = 10

for i in range(niters):
    print('-'*10 + ' iteration {} '.format(i) + '-'*10)  # print divider between iterations
    
    src_charges.real = np.random.random(nsrcs)     # generate new random charges
    src_charges.imag = np.random.random(nsrcs)          
    helmholtz.update_charges(tree, src_charges)      # update charges
    helmholtz.clear_values(tree)                     # clear values
    trg_values = helmholtz.evaluate(tree, fmm)       # evaluate potentials

    print("Error: ", fmm.verify(tree.leafs))       # check accuracy

---------- iteration 0 ----------
P2M                  : 7.0362100e-01
M2M                  : 3.7836200e-01
P2L                  : 1.1200000e-04
M2P                  : 2.5960000e-03
P2P                  : 4.6268300e-01
M2L                  : 1.8045100e+00
L2L                  : 3.3383000e-01
L2P                  : 7.7000300e-01
Error:  [6.582410878028779e-08, 1.0706826081656837e-06]
---------- iteration 1 ----------
P2M                  : 7.0671300e-01
M2M                  : 3.3134700e-01
P2L                  : 9.8000000e-05
M2P                  : 3.9000000e-05
P2P                  : 4.6973400e-01
M2L                  : 1.8259220e+00
L2L                  : 3.2764400e-01
L2P                  : 7.9791600e-01
Error:  [7.349900310250513e-08, 1.1672375514191189e-06]
---------- iteration 2 ----------
P2M                  : 8.5024400e-01
M2M                  : 3.7529800e-01
P2L                  : 9.5000000e-05
M2P                  : 5.6000000e-05
P2P                  : 4.7423000e-01
M2L      